In [2]:
import pandas.io.sql as psql
import pandas as pd
import pymysql
import re

In [16]:
REPORT_LIST_FILE = '/Users/PrincessYvonne/Desktop/sec.list.txt'

In [17]:
#Connect to Database
connection = pymysql.connect()

In [18]:
#Store Compustat data in Dataframe
sql = "SELECT tic, cusip, conm, datacqtr, atq, atq_change, atq_posneg, ceqq, ceqq_change, ceqq_posneg, oiadpq, oiadpq_change, oiadpq_posneg, saleq, saleq_change, saleq_posneg, cik FROM Compustat_Health_Quarterly_Sentiment"
df = psql.read_sql(sql, connection)
df['cik_period'] = df['cik'].map(str) + df['datacqtr'].map(str)
df.tail(10)

,tic,cusip,conm,datacqtr,atq,atq_change,atq_posneg,ceqq,ceqq_change,ceqq_posneg,oiadpq,oiadpq_change,oiadpq_posneg,saleq,saleq_change,saleq_posneg,cik,cik_period
114202,AHI.,G0132M121,AGILITY HEALTH INC,2013Q1,0.000,NA,NA,0.000,NA,NA,0.710,NA,NA,16.449,NA,NA\r,0,02013Q1
114203,AHI.,G0132M121,AGILITY HEALTH INC,2013Q2,0.000,NA,NA,0.000,NA,NA,0.720,1.408450704,pos,15.384,-6.474557724,neg\r,0,02013Q2
114204,AHI.,G0132M121,AGILITY HEALTH INC,2013Q3,0.000,NA,NA,0.000,NA,NA,0.080,-88.88888889,neg,13.582,-11.71346854,neg\r,0,02013Q3
114205,AHI.,G0132M121,AGILITY HEALTH INC,2013Q4,28.834,NA,NA,-8.332,NA,NA,0.848,960,pos,15.113,11.27227212,pos\r,0,02013Q4
114206,AHI.,G0132M121,AGILITY HEALTH INC,2014Q1,30.573,6.031074426,pos,-7.148,-14.21027364,neg,-0.305,-135.9669811,neg,15.343,1.52186859,pos\r,0,02014Q1
114207,AHI.,G0132M121,AGILITY HEALTH INC,2014Q2,28.947,-5.318418212,neg,-7.871,10.1147174,pos,0.859,-381.6393443,neg,15.808,3.030698038,pos\r,0,02014Q2
114208,AHI.,G0132M121,AGILITY HEALTH INC,2014Q3,29.357,1.416381663,pos,-10.407,32.21954008,pos,-0.341,-139.6973225,neg,15.541,-1.689018219,neg\r,0,02014Q3
114209,AHI.,G0132M121,AGILITY HEALTH INC,2014Q4,27.360,-6.802466192,neg,-12.803,23.02296531,pos,-0.373,9.384164223,pos,15.413,-0.823627823,neg\r,0,02014Q4
114210,AHI.,G0132M121,AGILITY HEALTH INC,2015Q1,28.381,3.731725146,pos,-12.651,-1.187221745,neg,0.834,-323.5924933,neg,15.768,2.303250503,pos\r,0,02015Q1
114211,AHI.,G0132M121,AGILITY HEALTH INC,2015Q2,28.126,-0.898488425,neg,-11.582,-8.449924907,neg,0.791,-5.1558753,neg,15.784,0.101471334,pos,0,02015Q2


In [6]:
sec_regex_CIK = re.compile(r'./(\d{4})_(Q\d{1})_(\d+)_.+_10-K_\d{4}-\d{2}-\d{2}')


In [7]:
# Open the SEC company file and read it as a list
with open(REPORT_LIST_FILE, 'r') as f:
    content = f.readlines()
content[:5]

['./1993_Q4_29082_DISNEY WALT CO_10-K_1993-12-22\n',
 './1993_Q4_32377_ELIZABETHTOWN GAS CO_10-K_1993-12-13\n',
 './1993_Q4_353944_INTERNATIONAL GAME TECHNOLOGY_10-K_1993-12-23\n',
 './1993_Q4_861439_AMERICAN MEDICAL HOLDINGS INC_10-K_1993-11-29\n',
 './1994_Q1_100240_TURNER BROADCASTING SYSTEM INC_10-K_1994-03-31\n']

In [8]:
# Apply the Regex matcher to all SEC companies to find the CIK
content_matched = list(map(lambda l: sec_regex_CIK.search(l), content))
content_matched[:5]

[<_sre.SRE_Match object; span=(0, 46), match='./1993_Q4_29082_DISNEY WALT CO_10-K_1993-12-22'>,
 <_sre.SRE_Match object; span=(0, 52), match='./1993_Q4_32377_ELIZABETHTOWN GAS CO_10-K_1993-12>,
 <_sre.SRE_Match object; span=(0, 62), match='./1993_Q4_353944_INTERNATIONAL GAME TECHNOLOGY_10>,
 <_sre.SRE_Match object; span=(0, 62), match='./1993_Q4_861439_AMERICAN MEDICAL HOLDINGS INC_10>,
 <_sre.SRE_Match object; span=(0, 63), match='./1994_Q1_100240_TURNER BROADCASTING SYSTEM INC_1>]

In [9]:
# Extract the CIK, Year, and Quarter from the Regex results
report_list = list(map(lambda m:(int(m.group(3)),int(m.group(1)), m.group(2), m.group(0)), content_matched))
print('Number of reports in the ZIP file: %d' % len(report_list))
report_list[:5]

Number of reports in the ZIP file: 153734


[(29082, 1993, 'Q4', './1993_Q4_29082_DISNEY WALT CO_10-K_1993-12-22'),
 (32377, 1993, 'Q4', './1993_Q4_32377_ELIZABETHTOWN GAS CO_10-K_1993-12-13'),
 (353944,
  1993,
  'Q4',
  './1993_Q4_353944_INTERNATIONAL GAME TECHNOLOGY_10-K_1993-12-23'),
 (861439,
  1993,
  'Q4',
  './1993_Q4_861439_AMERICAN MEDICAL HOLDINGS INC_10-K_1993-11-29'),
 (100240,
  1994,
  'Q1',
  './1994_Q1_100240_TURNER BROADCASTING SYSTEM INC_10-K_1994-03-31')]

In [10]:
#Convert Report List into DataFrame with similar columns as Compustat DataFrame to prepare for merging
df2= pd.DataFrame(report_list, columns=["cik", "year", "quarter", "filename"])  
df2['cik_period'] = df2['cik'].map(str) + df2['year'].map(str) + df2['quarter'].map(str)

df2.tail(5)

,cik,year,quarter,filename,cik_period
153729,1359555,2015,Q1,./2015_Q1_1359555_Pendrell Corp_10-K_2015-03-06,13595552015Q1
153730,775368,2015,Q1,./2015_Q1_775368_ALLEGHANY CORP -DE_10-K_2015-...,7753682015Q1
153731,921112,2015,Q1,./2015_Q1_921112_LIBERTY PROPERTY TRUST_10-K_2...,9211122015Q1
153732,1486597,2015,Q2,"./2015_Q2_1486597_SUPERNOVA ENERGY, INC._10-K_...",14865972015Q2
153733,1563699,2015,Q2,./2015_Q2_1563699_Covisint Corp_10-K_2015-05-27,15636992015Q2


In [14]:
df3 = pd.merge(df, df2, how='inner', on='cik_period')
df3.tail(10)

,tic,cusip,conm,datacqtr,atq,atq_change,atq_posneg,ceqq,ceqq_change,ceqq_posneg,...,oiadpq_posneg,saleq,saleq_change,saleq_posneg,cik_x,cik_period,cik_y,year,quarter,filename
12850,MODL,60689B107,MMODAL INC,2011Q1,414.262,-0.14871806,neg,47.608,8.350212795,pos,...,neg,111.236,0.635098703,pos\r,1441567,14415672011Q1,1441567,2011,Q1,./2011_Q1_1441567_MedQuist Holdings Inc._10-K_...
12851,MODL,60689B107,MMODAL INC,2012Q1,537.917,1.151195479,pos,126.451,1.119543226,pos,...,neg,117.400,1.127563722,pos\r,1441567,14415672012Q1,1441567,2012,Q1,./2012_Q1_1441567_MModal Inc._10-K_2012-03-15
12852,RPSE,76199U930,RESEARCH PHARMACL SVCS -REDH,2008Q1,0.000,NA,NA,0.000,NA,NA,...,NA,41.842,NA,NA\r,1420070,14200702008Q1,1420070,2008,Q1,./2008_Q1_1420070_RESEARCH PHARMACEUTICAL SERV...
12853,RPSE,76199U930,RESEARCH PHARMACL SVCS -REDH,2009Q1,0.000,-100,neg,0.000,-100,neg,...,pos,50.294,13.38458417,pos\r,1420070,14200702009Q1,1420070,2009,Q1,./2009_Q1_1420070_RESEARCH PHARMACEUTICAL SERV...
12854,RPSE,76199U930,RESEARCH PHARMACL SVCS -REDH,2010Q1,92.193,-4.226010534,neg,44.626,-3.628039563,neg,...,pos,64.710,3.926764635,pos\r,1420070,14200702010Q1,1420070,2010,Q1,./2010_Q1_1420070_RESEARCH PHARMACEUTICAL SERV...
12855,MDGN,58436Q203,MEDGENICS INC,2012Q1,5.201,-24.20577091,neg,0.535,-82.14881548,neg,...,neg,0.000,NA,NA\r,1138776,11387762012Q1,1138776,2012,Q1,"./2012_Q1_1138776_MEDGENICS, INC._10-K_2012-03-06"
12856,MDGN,58436Q203,MEDGENICS INC,2013Q1,33.659,336.7328403,pos,28.544,1375.90486,pos,...,pos,0.000,NA,NA\r,1138776,11387762013Q1,1138776,2013,Q1,"./2013_Q1_1138776_MEDGENICS, INC._10-K_2013-03-14"
12857,MDGN,58436Q203,MEDGENICS INC,2014Q1,20.251,-12.28396933,neg,15.594,-15.35580524,neg,...,neg,0.000,NA,NA\r,1138776,11387762014Q1,1138776,2014,Q1,"./2014_Q1_1138776_MEDGENICS, INC._10-K_2014-02-20"
12858,MDGN,58436Q203,MEDGENICS INC,2015Q1,27.412,-20.03500583,neg,23.899,-19.42889893,neg,...,pos,0.000,NA,NA\r,1138776,11387762015Q1,1138776,2015,Q1,"./2015_Q1_1138776_MEDGENICS, INC._10-K_2015-02-13"
12859,GIDYL,U3762T121,GI DYNAMICS INC,2015Q1,47.934,-19.43865546,neg,40.525,-19.23751445,neg,...,neg,0.616,30.23255814,pos\r,1245791,12457912015Q1,1245791,2015,Q1,"./2015_Q1_1245791_GI DYNAMICS, INC._10-K_2015-..."
